In [ ]:
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # Alias for "SEI porosity change"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",  # For compatibility with older PyBaMM versions
    }
)

# Set model parameters
param = pybamm.ParameterValues("OKane2022")

var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particles
    "r_p": 30,  # Positive electrode particles
}

# Set the maximum number of cycles
max_cycles = 2000  # You can increase it to 2000, but the computation will be intensive
cycle_numbers = [1] + list(range(50, max_cycles + 1, 50))  # Compute every 50 cycles
remaining_capacity_values = []  # Store remaining capacity

# Simulate step-by-step increasing cycle count
for k in cycle_numbers:
    print(f"Calculating cycle {k} ...")  # Show progress

    # Define experiment protocol
    exp_k = pybamm.Experiment(
        [
            "Hold at 4.2 V until C/100 (5 minute period)",
            "Rest for 4 hours (5 minute period)",
            "Discharge at 0.1C until 2.5 V (5 minute period)",  # Initial capacity check
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        ]
        + [
            (
                "Discharge at 1C until 2.5 V",  # Aging cycles
                "Charge at 0.3C until 4.2 V (5 minute period)",
                "Hold at 4.2 V until C/100 (5 minute period)",
            )
        ]
        * k
        + ["Discharge at 0.1C until 2.5 V (5 minute period)"],  # Final capacity check
    )

    # Run simulation
    sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
    sol_k = sim_k.solve()

    # Extract LLI loss (unit: A.h) and calculate Remaining Capacity (%)
    Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600  # Unit conversion
    remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100  # Calculate remaining capacity percentage
    remaining_capacity_values.append(remaining_capacity_k)  # Store data

# Plot Remaining Capacity (%) vs. Cycle Number (k)
plt.figure(figsize=(8, 6))
plt.plot(cycle_numbers, remaining_capacity_values, label="Remaining Capacity (%)", color="blue", linewidth=2)
plt.xlabel("Number of cycles (k)")
plt.ylabel("Remaining Capacity (%)")
plt.title("Remaining Capacity vs. Cycle Number")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)  # Limit y-axis range from 0% to 100%
plt.show()

# Output remaining capacity for the last cycle
print(f"When k={cycle_numbers[-1]}, the remaining capacity is: {remaining_capacity_values[-1]:.2f}%")
